In [3]:
from utils.datautils import prepare_data, create_vocab
prepare_data('train_pos_full.txt', 'train_neg_full.txt', 'data')

(134000, 2)
(66000, 2)


In [4]:
VOCAB, MAX_LEN = create_vocab('data')

In [5]:
EMBEDDING_DIM = 100
K = 200
BATCH_SIZE = 64

In [6]:
from utils.glove import load_glove
word_to_idx, embeddings, PAD_IDX, words = load_glove('embeddings/glove/glove.twitter.27B.100d.txt', EMBEDDING_DIM)

In [9]:
VOCAB_SIZE = len(embeddings)
VOCAB_SIZE

1193515

In [10]:
from datasets.glove_dataset import GloveDataset

train_data = GloveDataset('data/train.csv', MAX_LEN, word_to_idx, PAD_IDX, VOCAB)
valid_data = GloveDataset('data/valid.csv', MAX_LEN, word_to_idx, PAD_IDX, VOCAB)

In [13]:
# train_data.__len__(), valid_data.__len__()
train_data.__getitem__(778)
        

(array([    506,     293,      10,     334,      49,      10,      36,
            187,     369,      49,     210,      51,      76,      49,
            456,      49,      11,      73,      10,      36,     456,
             51,     351,     187,      50,     236,      11,     187,
            210,     369,     506,     210,      11,     187,      11,
            293,      47,     137,     187,      76,      11,     210,
            151,     137,     187,      11,     456,      49,     187,
            369,      73,      50,      51,     456,     369,      50,
             51,     187,     187,     369,      49,     199,      11,
             47,     236,     369,      10,     293,      49,      10,
             51,      36,     199,      49,      73,     236,      73,
             10,     187,      49,      10,      36,     137,      51,
             73,      11,      36,     210,      49,      71,      51,
             73,     293,      60, 1193514, 1193514, 1193514, 1193514,
      

In [ ]:
from torch.utils.data import DataLoader
training_params = {"batch_size": BATCH_SIZE,
                       "shuffle": True,
                       "drop_last": True}

validation_params = {"batch_size": BATCH_SIZE,
                      "shuffle": False,
                      "drop_last": True}
train_iterator = DataLoader(train_data, **training_params)
valid_iterator = DataLoader(valid_data, **validation_params)

In [ ]:
from models.SIMPLERNN import RNN as RNN

In [ ]:
INPUT_DIM = VOCAB_SIZE # len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = PAD_IDX # :)

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [ ]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
import torch 
model.embedding.weight.data.copy_(torch.Tensor(embeddings))

"<user> i dm'd you what to do get your followers to follow you on my your new twitter . where is this shindig ? x\n"

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch.optim as optim
import torch.nn as nn

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

(200000, 2)


,Tweet,Label
156366,broadways fabulous phantoms ( audio cd the cd ...,0
70313,<user> <user> coil idea piano's everywhere .\n,1
133644,with <user> while missing <user>\n,0
172934,alltel ppc 6700 directsync - mobile charging k...,0
6249,only 100 days until the london olympics ... di...,1


In [ ]:
from utils.mterics import AverageMeter, binary_accuracy

1    100000
0    100000
Name: Label, dtype: int64

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    num_iter_per_epoch = len(iterator)
    losses = AverageMeter()
    accuracies = AverageMeter()
    model.train()
    iter = 0
    for batch in iterator:
        iter += 1
        optimizer.zero_grad()
        x_wrd, lengths, labels = batch
        x_wrd = torch.LongTensor(x_wrd)
        if torch.cuda.is_available():
            x_wrd = x_wrd.cuda()
            labels = labels.cuda()
        
        predictions = model(x_wrd).squeeze(1)
        
        loss = criterion(predictions, labels.float())
        
        acc = binary_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        losses.update(loss.data, x_wrd.size(0))
        accuracy = binary_accuracy(predictions, labels)
        accuracies.update(accuracy, x_wrd.size(0))

        if (iter % 20 == 0) and (iter > 0):
                print("[Train - Epoch: {}] , Iteration: {}/{} , Loss: {}, Accuracy: {}".format(
                epoch + 1,
                iter,
                num_iter_per_epoch,
                losses.avg,
                accuracies.avg
            ))
        
    return losses.avg.item(), accuracies.avg.item()

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    losses = AverageMeter()
    accuracies = AverageMeter()
    num_iter_per_epoch = len(iterator)

    iter = 0
    for batch in iterator:
        iter += 1
        x_wrd, lengths, labels = batch
        x_wrd = torch.LongTensor(x_wrd)
        if torch.cuda.is_available():
            x_wrd = x_wrd.cuda()
            labels = labels.cuda()

        with torch.no_grad():
            predictions = model(x_wrd).squeeze(1)
        
        loss = criterion(predictions, labels.float())
        accuracy = binary_accuracy(predictions, labels)
        losses.update(loss.data, x_wrd.size(0))
        accuracies.update(accuracy, x_wrd.size(0))

        if (iter % 20 == 0) and (iter > 0):
            print("[Validation - Epoch: {}] , Iteration: {}/{} , Loss: {}, Accuracy: {}".format(
                epoch + 1,
                iter,
                num_iter_per_epoch,
                losses.avg,
                accuracies.avg
            ))


    return losses.avg.item(), accuracies.avg.item()

In [ ]:

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:

N_EPOCHS = 15

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model-lstm.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
from utils.datautils import prepare_test_data

prepare_test_data('test_data.txt', 'data')
test_iterator = GloveDataset('data/test.csv', MAX_LEN, word_to_idx, PAD_IDX, VOCAB)

In [ ]:
def predict(model, iterator):
    model.eval()
    output_arr = []
    id = 1
    for tweet in iterator: 
        x_wrd, lengths, label = tweet
        x_wrd = torch.LongTensor(x_wrd).unsqueeze(0)
        lengths = torch.LongTensor([lengths])
        if torch.cuda.is_available():
            x_wrd = x_wrd.cuda()
            lenghts = lengths.cuda()
        prediction = torch.sigmoid(model(x_chr, x_wrd, lengths))
        # print()
        if int(torch.round(prediction).item()) == 0:
            output_arr.append([id, -1])
        elif int(torch.round(prediction).item()) == 1:
            output_arr.append([id, 1])
        else:
            raise Exception("This should never happen")
        id += 1

        if id % 1000 == 0:
            print('Predicting example {}/{}'.format(id, len(iterator)))
    
    return output_arr


In [ ]:
pred = predict(model, test_iterator)

In [ ]:
from utils.datautils import write_predictions
pred_df = write_predictions(pred, 'Simple-RNN.csv', data_dir='predfiles')